In [27]:
import requests
from bs4 import BeautifulSoup
import json


In [28]:
from currency_converter import CurrencyConverter, SINGLE_DAY_ECB_URL
cc = CurrencyConverter(SINGLE_DAY_ECB_URL)

In [29]:
x, curr = 5000, 'SEK'

print(round(cc.convert(x, curr), 3), round(cc.convert(x, curr)))

448.712 449


In [30]:
x, curr = 1500, 'NOK'

print(round(cc.convert(x, curr), 3), round(cc.convert(x, curr)))

135.612 136


In [69]:
# from_to = "Gdansk/Bodo"
from_to = "Frankfurt am Main, Germany/Moscow, Russia"
#from_to = 'London/Milan'

In [70]:
base_url = "https://www.rome2rio.com/map/"
    
tmp_url = base_url + from_to  
s = requests.session()
r = s.get(tmp_url, )
print(r.status_code, str(tmp_url))

200 https://www.rome2rio.com/map/Frankfurt am Main, Germany/Moscow, Russia


In [ ]:
soup = BeautifulSoup(r.text, "html.parser")
print(soup.prettify())

In [72]:
# seo-data extraction and recording to the output json file
seo_data = dict()
tag_of_interest = soup.find_all('script', type='application/ld+json')
for item in tag_of_interest:
    for elem in item.contents:

        data = json.loads(elem)
        try:
                
            for i, data_item in enumerate(data['mainEntity'], start=1):
                
                seo_data[i] = {'question': data_item['name'], 
                               'answer': data_item['acceptedAnswer']['text'][:str.find(data_item['acceptedAnswer']['text'], ' <')]}
        except:
            continue
print(json.dumps(seo_data, indent=4))

# with open(f"{from_to.split('/')[0]}-{from_to.split('/')[1]}-seo_data.json", 'w') as file:
    # json.dump(seo_data, file, indent=4)

{
    "1": {
        "question": "What is the most cost-effective way to get from Frankfurt am Main to Moscow?",
        "answer": "The cost-effective way to get from Frankfurt am Main to Moscow is to bus via \u0141om\u017ca, which costs \u20ac80&nbsp;-&nbsp;\u20ac110 and takes 45h 25m."
    },
    "2": {
        "question": "What is the fastest way to get from Frankfurt am Main to Moscow?",
        "answer": "The quickest way to get from Frankfurt am Main to Moscow is to fly which costs \u20ac90&nbsp;-&nbsp;\u20ac310 and takes 9h 33m."
    },
    "3": {
        "question": "How far is it from Frankfurt am Main to Moscow?",
        "answer": "The distance between Frankfurt am Main and Moscow is 2021 km. The road distance is 2334.5 km."
    },
    "4": {
        "question": "How do I travel from Frankfurt am Main to Moscow without a car?",
        "answer": "The best way to get from Frankfurt am Main to Moscow without a car is to train and bus which takes 33h 53m and costs \u20ac170&nbs

In [73]:
# finds common info about the trip
title = soup.find('meta', property="og:title")
print(title['content'])

Frankfurt am Main to Moscow - 10 ways to travel via train, plane, and bus


In [74]:
# gets quickest and cheapest ways
description = soup.find('meta', property="og:description")
print(description['content'])

The cheapest way to get from Frankfurt am Main to Moscow costs only RUB 7530, and the quickest way takes just 9½ hours. Find the travel option that best suits you.


In [75]:
# finds nodes geolocation info
nodes = soup.find('meta', id="deeplinkNodes")
nodes['content']

'[["GeoCoderResultV1","Frankfurt am Main","Frankfurt am Main, Germany",50.11552,8.68417,"City","DE","Frankfurt-am-Main","False",2.5],["GeoCoderResultV1","Moscow","Moscow, Russia",55.75222,37.61556,"Capital","RU","Moscow","False",2.5]]'

In [76]:
# find trip`s pathes
pathes = soup.find('meta', id="deeplinkTrip")
pathes = json.loads(pathes['content'])[2][1]

In [77]:
# calculates number of pathes
num_pathes = len([i for i in pathes])
print(f'{num_pathes = }')

num_pathes = 10


In [78]:
# get all pathes wirh numbers
for i, item in enumerate(pathes):
    print('path ', i, '\t', item, '\n\n')

path  0 	 [0, ['node', 'Frankfurt am Main', 50.11552, 8.68417, 'DE', 'Europe/Berlin', 2.5], ['node', 'Moscow', 55.75222, 37.61556, 'RU', 'Europe/Moscow', 2.5], 'Fly-Frankfurt-to-Moscow-Sheremetyevo', 'Fly Frankfurt to Moscow Sheremetyevo', 34380, 26280, 8100, [['transit', 'train', 'purple', 900, 531.459, 12.97911, ['station', 'Frankfurt', 50.11406, 8.678139, 'DE', 'Europe/Berlin'], ['station', 'Frankfurt', 50.05132, 8.571101, 'DE', 'Europe/Berlin'], '', 'False', [900, 12.97911, ['station', 'Frankfurt', 50.11406, 8.678139, 'DE', 'Europe/Berlin'], ['station', 'Frankfurt', 50.05132, 8.571101, 'DE', 'Europe/Berlin'], 'train', ['Commuter', ' Bicycles', ' Wheelchair'], 'S9', 569, [['Deutsche Bahn Regional', '//static.r2r.io/logos/Trains/96/_DEUTSCHE_BAHN_REGIONAL_201908031415_201908031421.png', 'https://www.bahn.com/i/view/index.shtml', 'bahn.com', 900, 12.97911, 569, 'S8, S9', [['https://reiseauskunft.bahn.de/bin/query.exe/en?revia=yes&existOptimizePrice=1&country=overseas&initialAvail=ON&S

In [ ]:
# unpacking all routes from each path
for path_id, path in enumerate(pathes):
    for i, route in enumerate(path):
        print(path_id, '\t', i, '\t', route, '\n')

In [80]:
# unpacking each route's info from the section 8 (except last one [:-1], because it is 'hotel')
for path_id, path in enumerate(pathes):
    print(f'path # {path_id}\n')
    for i, route in enumerate(path[8]):
        for j, way in enumerate(route):
            print(path_id, '\t', i, '\t', j, '\t', way, '\n')

path # 0

0 	 0 	 0 	 transit 

0 	 0 	 1 	 train 

0 	 0 	 2 	 purple 

0 	 0 	 3 	 900 

0 	 0 	 4 	 531.459 

0 	 0 	 5 	 12.97911 

0 	 0 	 6 	 ['station', 'Frankfurt', 50.11406, 8.678139, 'DE', 'Europe/Berlin'] 

0 	 0 	 7 	 ['station', 'Frankfurt', 50.05132, 8.571101, 'DE', 'Europe/Berlin'] 

0 	 0 	 8 	  

0 	 0 	 9 	 False 

0 	 0 	 10 	 [900, 12.97911, ['station', 'Frankfurt', 50.11406, 8.678139, 'DE', 'Europe/Berlin'], ['station', 'Frankfurt', 50.05132, 8.571101, 'DE', 'Europe/Berlin'], 'train', ['Commuter', ' Bicycles', ' Wheelchair'], 'S9', 569, [['Deutsche Bahn Regional', '//static.r2r.io/logos/Trains/96/_DEUTSCHE_BAHN_REGIONAL_201908031415_201908031421.png', 'https://www.bahn.com/i/view/index.shtml', 'bahn.com', 900, 12.97911, 569, 'S8, S9', [['https://reiseauskunft.bahn.de/bin/query.exe/en?revia=yes&existOptimizePrice=1&country=overseas&initialAvail=ON&S=Frankfurt&Z=Frankfurt&start=1', 'Deutsche Bahn Regional', 'Website', 'Deutsche Bahn Regional', '', '', '', '', '']], [

In [ ]:
# unpacking the prices (min, mid, max) of the path from the section 20
for path_id, path in enumerate(pathes, start=1):
    print(path_id, '\n\t', path[20], '\n')

In [ ]:
# unpacking the prices (min, mid, max) for each route of the path from the section 20
for path_id, path in enumerate(pathes, start=1):
    for i, route in enumerate(path[20]):
        print(path_id, '\n\t', i, '\t', route, '\n')